<a href="https://colab.research.google.com/github/changhorang/SSAC_study/blob/main/DL3%20(PyTorch)/%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC/Chapter4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter4. 문서에 꼬리표 달기

## 4-1 문서 분류 모델 훑어보기
- 문서분류: 주어진 문서의 범주를 분류하는 과제 (긍/부정, 정치/경제/연예...)
- 네이버 영화 리뷰 말뭉치(NSMC) 이용

### 모델 구조
- CLS, SEP의 시작과 끝을 알리는 토큰을 기존 토큰 앞뒤에 추가
- BERT 모델에 토큰 입력 후 pooler_output(문장 수준의 벡터) 출력
- 추가 모듈을 덧붙여 긍정/부정 예측

### 태스크 모듈
- pooler_output 벡터에 dropout 적용
- pooler_output => 분류해야할 범주의 수의 차원을 갖는 벡터로 변환
- softmax 적용
- 최종 예측값과 target을 비교해 모델 전체를 업데이트 (*파인튜닝)

## 4-2 문서뷴류 모델 학습하기
### 영화 리뷰 감성 분석 모델 만들기
#### 각종 설정하기

In [1]:
!pip install ratsnlp

     |████████████████████████████████| 53 kB 1.7 MB/s 
     |████████████████████████████████| 2.8 MB 13.5 MB/s 
     |████████████████████████████████| 806 kB 91.0 MB/s 
     |████████████████████████████████| 57 kB 5.7 MB/s 
     |████████████████████████████████| 332 kB 100.8 MB/s 
     |████████████████████████████████| 636 kB 75.6 MB/s 
     |████████████████████████████████| 132 kB 97.6 MB/s 
     |████████████████████████████████| 829 kB 89.4 MB/s 
     |████████████████████████████████| 3.3 MB 85.8 MB/s 
     |████████████████████████████████| 61 kB 677 kB/s 
     |████████████████████████████████| 895 kB 78.6 MB/s 
     |████████████████████████████████| 1.1 MB 63.7 MB/s 
     |████████████████████████████████| 96 kB 8.2 MB/s 
     |████████████████████████████████| 192 kB 76.4 MB/s 
     |████████████████████████████████| 271 kB 78.2 MB/s 
     |████████████████████████████████| 160 kB 76.9 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070

In [2]:
import torch
from ratsnlp.nlpbook.classification import ClassificationTrainArguments

args = ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base',
                                    downstream_corpus_name='nsmc',
                                    downstream_model_dir='/content/drive/MyDrive/nlpbook/checkpoint-doccls',
                                    batch_size=32 if torch.cuda.is_available() else 4,
                                    learning_rate=5e-5,
                                    max_seq_length=128,
                                    epochs=3,
                                    tpu_cores=0 if torch.cuda.is_available() else 8,
                                    seed=7)

In [3]:
from ratsnlp import nlpbook

nlpbook.set_seed(args)

set seed: 7


In [4]:
# 코드 실행해 각종 로그를 출력하는 로거 설정
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='document-classification', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/root/Korpora', downstream_model_dir='/content/drive/MyDrive/nlpbook/checkpoint-doccls', max_seq_length=128, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, cpu_workers=4, fp16=False, tpu_cores=0)


#### 말뭉치 내려받기

In [5]:
from Korpora import Korpora

Korpora.fetch(corpus_name=args.downstream_corpus_name,
              root_dir=args.downstream_corpus_root_dir,
              force_download=True)

[nsmc] download ratings_train.txt: 14.6MB [00:00, 155MB/s]
[nsmc] download ratings_test.txt: 4.90MB [00:00, 55.8MB/s]


#### 토크나이저 준비하기

In [6]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained(args.pretrained_model_name, do_lower_case=False)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

#### Data Preprocessing

In [7]:
from ratsnlp.nlpbook.classification import NsmcCorpus, ClassificationDataset

corpus = NsmcCorpus()
# pytorch의 dataloader와 동일한 기능
train_dataset = ClassificationDataset(args=args, 
                                      corpus=corpus,
                                      tokenizer=tokenizer,
                                      mode='train')

INFO:ratsnlp:Creating features from dataset file at /root/Korpora/nsmc
INFO:ratsnlp:loading train data... LOOKING AT /root/Korpora/nsmc/ratings_train.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 34.387 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 아 더빙.. 진짜 짜증나네요 목소리
INFO:ratsnlp:tokens: [CLS] 아 더 ##빙 . . 진짜 짜증나네 ##요 목소리 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

In [8]:
corpus.num_labels

2

- ClassificationDataset
    - NsmcCorpus에서 넘긴 문장과 레이블을 각각 tokenizer를 활용해 모델이 학습할 수 있는 형태(ClassificationFeatures)로 가공

- ClassificationFeatures
    - input_ids: index로 변환된 토큰 시퀀스 (list_int)
    - attention_mask: 해당 토큰이 패딩(0)인지 아닌지(1) 표현 (list_int)
    - token_type_ids: segment 정보 (list_int)
    - label: 정수로 바뀐 레이블 정보 (int)

- ClassificationFeatures.token_type_ids: BERT 모델의 경우 pretrain과제가 빈칸 맞히기와 이어진 문서 맞히기 (next sentence prediction, 2개의 문서가 이어진지 아닌지 ***이진분류***) 수행함. 여기서 token sequence가 0이면 첫번째 문서, 1이면 두번째 문서를 의미하는 정보 

In [9]:
# Example
train_dataset[0]

ClassificationFeatures(input_ids=[2, 2170, 832, 5045, 17, 17, 7992, 29734, 4040, 10720, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [10]:
from torch.utils.data import DataLoader, RandomSampler

train_dataloader = DataLoader(train_dataset,
                              batch_size=args.batch_size,
                              sampler=RandomSampler(train_dataset, replacement=False), # 비복원(False) 랜덤 추출
                              collate_fn=nlpbook.data_collator, # 인스턴스들을 종류별로 모아 tensor로 변경
                              drop_last=False,
                              num_workers=args.cpu_workers)

In [11]:
# valid
from torch.utils.data import SequentialSampler

val_dataset = ClassificationDataset(args=args,
                                    corpus=corpus,
                                    tokenizer=tokenizer,
                                    mode='test')

val_dataloader = DataLoader(val_dataset,
                            batch_size=args.batch_size,
                            sampler=SequentialSampler(val_dataset),
                            collate_fn=nlpbook.data_collator,
                            drop_last=False,
                            num_workers=args.cpu_workers)

INFO:ratsnlp:Creating features from dataset file at /root/Korpora/nsmc
INFO:ratsnlp:loading test data... LOOKING AT /root/Korpora/nsmc/ratings_test.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 11.309 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 굳 ㅋ
INFO:ratsnlp:tokens: [CLS] 굳 ㅋ [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

#### 모델 불러오기
- BertForSequenceClassification: pretrained BERT모델 위에 분류용 태스크 모듈이 붙어 있는 class

In [12]:
from transformers import BertConfig, BertForSequenceClassification

pretrained_model_config = BertConfig.from_pretrained(args.pretrained_model_name,
                                                     num_labels=corpus.num_labels) # 2

model = BertForSequenceClassification.from_pretrained(args.pretrained_model_name,
                                                      config=pretrained_model_config)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

#### 모델학습시키기
- 파이토치 라이트닝에서 제공하는 LightningModule 클래스를 상속받아 task 정의
- task에는 모델, 옵티마이저, 학습 과정 등이 정의

In [13]:
from ratsnlp.nlpbook.classification import ClassificationTask

task = ClassificationTask(model, args)
trainer = nlpbook.get_trainer(args)

trainer.fit(task, train_dataloader=train_dataloader, val_dataloaders=val_dataloader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | BertForSequenceClassification | 108 M 
--------------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
435.680   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

- ClassificationTask: optimizer(AdamW), learning scheduler (ExponentialLR) 이용
    - ExponentialLR: 현재 에포크의 lr를 이전에포크의 lr*gamma(0.9)로 스케줄링

    - https://ratsgo.github.io/nlpbook/docs/doc_cls/detail 참고

## 4-3 학습 마친 모델을 실전 투입하기
- 인퍼런스(inference): 학습을 마친 모델로 실제 과제를 수행하는 행위나 과정을 의미

### 영화 리뷰 감성 분석 웹서비스 만들기
#### 환경 설정하기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install ratsnlp

     |████████████████████████████████| 53 kB 2.0 MB/s 
     |████████████████████████████████| 2.8 MB 51.5 MB/s 
     |████████████████████████████████| 57 kB 6.2 MB/s 
     |████████████████████████████████| 806 kB 45.5 MB/s 
     |████████████████████████████████| 636 kB 38.9 MB/s 
     |████████████████████████████████| 332 kB 55.1 MB/s 
     |████████████████████████████████| 829 kB 54.8 MB/s 
     |████████████████████████████████| 132 kB 56.9 MB/s 
     |████████████████████████████████| 895 kB 45.2 MB/s 
     |████████████████████████████████| 61 kB 548 kB/s 
     |████████████████████████████████| 3.3 MB 47.4 MB/s 
     |████████████████████████████████| 1.1 MB 51.2 MB/s 
     |████████████████████████████████| 96 kB 5.5 MB/s 
     |████████████████████████████████| 271 kB 46.0 MB/s 
     |████████████████████████████████| 160 kB 62.1 MB/s 
     |████████████████████████████████| 192 kB 60.6 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 

In [15]:
from ratsnlp.nlpbook.classification import ClassificationDeployArguments

args = ClassificationDeployArguments(pretrained_model_name='beomi/kcbert-base',
                                     downstream_model_dir='/content/drive/MyDrive/nlpbook/checkpoint-doccls',
                                     max_seq_length=128)

downstream_model_checkpoint_fpath: /content/drive/MyDrive/nlpbook/checkpoint-doccls/epoch=0-val_loss=0.27.ckpt


#### 토크나이저 및 모델 불러오기

In [16]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained(args.pretrained_model_name, do_lower_case=False)

In [18]:
# fine tuning한 모델 불어오기
import torch
from transformers import BertConfig
from transformers import BertForSequenceClassification

# checkpoint load
fine_tuned_model_ckpt = torch.load(args.downstream_model_checkpoint_fpath, map_location=torch.device('cuda'))

# BERT setting load
pretrained_model_config = BertConfig.from_pretrained(args.pretrained_model_name,
                                                     num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel())

# model 초기화
model = BertForSequenceClassification(pretrained_model_config)

# checkpoint 주입
model.load_state_dict({k.replace('model.', ''): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})

# evaluation 전환
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

#### 모델 출력값 만들고 후처리하기 (inference 함수)
- 문장 토큰화
- input_ids, attention_mask, token_type_ids 생성 후 tensor 변환
- 모델에 입력하기
- output.logits (모델 출력값)을 softmax 함수를 이용해 긍정/부정 확률로 출력
- 최대값에 따라 긍정/부정을 출력하는 pred값 만들기

In [19]:
# inference 함수
def inference_fn(sentence):
    """
    sentence: 긍정 및 부정을 예측하고 싶은 문장 (input)
    """
    # 토큰화 및 input_ids, attention_mask, token_type_ids 생성
    inputs = tokenizer([sentence], 
                       max_length=args.max_seq_length,
                       padding='max_length',
                       truncation=True)
    
    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})
        prob = outputs.logits.softmax(dim=1)
        positive_prob = round(prob[0][1].item(), 4)
        negative_prob = round(prob[0][0].item(), 4)
        pred = '긍정 (positive)' if torch.argmax(prob) == 1 else '부정 (negative)'

    return {
        'sentence': sentence,
        'prediction': pred,
        'positive_data': f'긍정 {positive_prob}',
        'negative_data': f'부정 {negative_prob}',
        'positive_width': f'{positive_prob*100}%',
        'negative_width': f'{negative_prob*100}%'
    }


In [20]:
inference_fn('핵 꿀잼. 한 번 더 보러간다!ㅋㅋㅋ')

{'negative_data': '부정 0.0201',
 'negative_width': '2.01%',
 'positive_data': '긍정 0.9799',
 'positive_width': '97.99%',
 'prediction': '긍정 (positive)',
 'sentence': '핵 꿀잼. 한 번 더 보러간다!ㅋㅋㅋ'}

#### 웹서비스 (flask)

In [21]:
from ratsnlp.nlpbook.classification import get_web_service_app

app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app "ratsnlp.nlpbook.classification.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://73d6-34-83-237-206.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [31/Dec/2021 06:13:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/Dec/2021 06:13:59] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [31/Dec/2021 06:14:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/Dec/2021 06:14:43] "GET /favicon.ico HTTP/1.1" 404 -
